# Notebook to parse tRNA alignments from HTML page gtRNAdb

working version
requires manual editing/removing of a duplicate ```RZ9990_NCA_HUMAN``` entry at the end of the file



In [1]:
import requests
import sys

from bs4 import BeautifulSoup

In [2]:
url = "http://gtrnadb.ucsc.edu/genomes/eukaryota/Hsapi38/Hsapi38-align.html"

out_stockholm_fn = "hs38_gtrnadb_algn_01.sto"

align_header = "# STOCKHOLM 1.0"

In [3]:
def parse_seq_alignments(alg_list):
    counter = 0
    old_id_prefix = ""
    old_anticodon = ""
    for alig in alg_list:
        lines = alig.text.split("\n")
        for line in lines:
            if line.find(">>") == -1 and line.find("filtered") == -1:
                sl = line.split()
                if len(sl) != 0:
                    #print(line)
                    seq, seq_id = sl[:2]
                    if seq_id.startswith("tRNA"):
                        split_id = seq_id.split("-")
                        id_prefix = "-".join(split_id[:3])
                        try:
                            anticodon = split_id[2]
                        except:
                            break
                            # there is an error in the HTML before the tRNA_Sup

                        if id_prefix != old_id_prefix:
                            if counter != 0:
                                print("//")
                            counter += 1
                            print(align_header)
                            align_group_id = f"#=GF ID {id_prefix}"
                            print(align_group_id)

                        out_str = f"{seq_id}\t{seq}"
                        print(out_str)
                        old_id_prefix = id_prefix
                    else:
                        split_id = seq_id.split("_")
                        anticodon = split_id[1]
                        short_id = f"{split_id[0]}_{split_id[1]}_HUMAN"
                        out_str = f"{short_id}\t{seq}"
                        print(out_str)

    print("//")    


In [4]:
# get the HTML

r = requests.get(url)
soup = BeautifulSoup(r.text)

# get all alignments from html
alig_list = soup.find_all('div', attrs={'class':'seq_alignment'})

#output in the browser
parse_seq_alignments(alig_list)


# STOCKHOLM 1.0
#=GF ID tRNA-Ala-AGC
tRNA-Ala-AGC-1-1	GGGGGTATAGCTCAGT.GGTA.GAGCGCGTGCTTAGCATGCACGAG...........GtCCTGGGTTCGATCCCCAGTACCTCCA...
tRNA-Ala-AGC-2-1	GGGGGTGTAGCTCAGT.GGTA.GAGCGCGTGCTTAGCATGCACGAG...........GcCCCGGGTTCAATCCCCGGCACCTCCA...
tRNA-Ala-AGC-2-2	GGGGGTGTAGCTCAGT.GGTA.GAGCGCGTGCTTAGCATGCACGAG...........GcCCCGGGTTCAATCCCCGGCACCTCCA...
tRNA-Ala-AGC-7-1	GGGGGTGTAGCTCAGT.GGTA.GAGCGCGTGCTTAGCATGCACGAG...........GcCCCGGGTTCAATCCCCGGCACCTCCA...
tRNA-Ala-AGC-5-1	GGGGGTGTAGCTCAGT.GGTA.GAGCGCGTGCTTAGCATGCACGAG...........GcCCTGGGTTCAATCCCCAGCACCTCCA...
tRNA-Ala-AGC-3-1	GGGGGTGTAGCTCAGT.GGTA.GAGCGCGTGCTTAGCATGTACGAG...........GtCCCGGGTTCAATCCCCGGCACCTCCA...
tRNA-Ala-AGC-4-1	GGGGATGTAGCTCAGT.GGTA.GAGCGCATGCTTAGCATGCATGAG...........GtCCCGGGTTCGATCCCCAGCATCTCCA...
tRNA-Ala-AGC-6-1	GGGGGTATAGCTCAGC.GGTA.GAGCGCGTGCTTAGCATGCACGAG...........GtCCTGGGTTCAATCCCCAATACCTCCA...
tRNA-Ala-AGC-10-1	GGGGAATTAGCTCAAGTGGTA.GAGCGCTTGCTTAGCACGCAAGAG...........GtAGTGGGATCGATGCCCACATTCTCCA...
tRNA-Ala

In [5]:
#output in the external file

saveout = sys.stdout
output_fh = open(out_stockholm_fn, "w")
sys.stdout = output_fh

parse_seq_alignments(alig_list)

sys.stdout = saveout
output_fh.close()